In [ ]:
import psycopg2
from psycopg2.extras import execute_batch
import os
import shutil
from ultralytics import YOLO

# Set database connection environment variables
os.environ['DATABASE_NAME'] = 'mywardrobe'
os.environ['DATABASE_USER'] = 'postgres'
os.environ['DATABASE_PASSWORD'] = 'mysecretpassword'
os.environ['DATABASE_HOST'] = 'my-postgres'  # Change to 'localhost' if directly connected to PostgreSQL without Docker

# Database connection parameters
db_params = {
    "dbname": os.getenv("DATABASE_NAME"),
    "user": os.getenv("DATABASE_USER"),
    "password": os.getenv("DATABASE_PASSWORD"),
    "host": os.getenv("DATABASE_HOST")
}

# Connect to your PostgreSQL database
conn = psycopg2.connect(**db_params)
cur = conn.cursor()

# Create tables for each type of clothing
clothing_types = ['bag', 'dress', 'hat', 'jacket', 'pant', 'shirt', 'shoe', 'short', 'skirt']
for clothing_type in clothing_types:
    cur.execute(f"""
    CREATE TABLE IF NOT EXISTS {clothing_type} (
        id SERIAL PRIMARY KEY,
        image_url VARCHAR(255)
    );
    """)
conn.commit()

# Function to insert items into the database, modified to handle different tables
def insert_items(clothing_type, items):
    query = f"INSERT INTO {clothing_type} (image_url) VALUES (%s);"
    execute_batch(cur, query, [(item,) for item in items])
    conn.commit()

# Load pre-trained model
detection_model = YOLO("best.pt")

# Directory containing the images
images_directory = 'ClothingRecommendation/images/'
crop_directory = 'runs/detect/predict'

# Ensure the target crops folder exists
target_crops_folder = 'crops'
if not os.path.exists(target_crops_folder):
    os.makedirs(target_crops_folder)

# Process images and move crops to a common folder, independent of runs
image_files = [f for f in os.listdir(images_directory) if f.endswith('.jpg')]
for image_file in image_files:
    image_path = os.path.join(images_directory, image_file)
    predictions = detection_model.predict(source=image_path, conf=0.5, save=True, line_width=2, show_labels=True, save_crop=True)

    # New logic to handle common crops folder
    for clothing_type in clothing_types:
        source_crops_folder = os.path.join(target_crops_folder, clothing_type)
        source_crops_folder_item = os.path.join(crop_directory, source_crops_folder)
        if os.path.exists(source_crops_folder_item):
            items_to_insert = []
            for crop_img in os.listdir(source_crops_folder_item):
                source_path = os.path.join(source_crops_folder_item, crop_img)
                destination_path = os.path.join(target_crops_folder, clothing_type, crop_img)
                if not os.path.exists(os.path.dirname(destination_path)):
                    os.makedirs(os.path.dirname(destination_path))
                shutil.move(source_path, destination_path)
                items_to_insert.append(destination_path)

            # Enter into type of clothe the given clothe
            if items_to_insert:
                insert_items(clothing_type, items_to_insert)

    # Remove predict directory after entering all the predict crop in my database            
    if os.path.exists(crop_directory):
        shutil.rmtree(crop_directory)

# Close the database connection
cur.close()
conn.close()


In [ ]:
# Connect to your PostgreSQL database
conn = psycopg2.connect(**db_params)
cur = conn.cursor()

# Query to fetch all table names in the 'public' schema of your database
cur.execute("""
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public'
ORDER BY table_name;
""")

# Fetch all rows from the query
tables = cur.fetchall()

# Print the names of the tables and their data
for table in tables:
    table_name = table[0]  # Correctly extract the table name from the tuple
    try:
        cur.execute(f"SELECT * FROM {table_name};")  # Use the extracted table name in the query
        rows = cur.fetchall()
        print(f"Data from table: {table_name}")
        for row in rows:
            print(row)
    except Exception as e:
        print(f"An error occurred: {e}")

# Close the database connection
cur.close()
conn.close()
